# split

> Fill in a module description here

In [11]:
#| default_exp split

In [12]:
#| hide
from nbdev.showdoc import *

In [1]:
#| export
import subprocess
from fastai.vision.all import *
from tqdm import tqdm
import argparse
import numpy as np

### Data record
> ```
temset/
├── videos/
│   ├── surg-001/
│   │   ├── video1.mp4
│   │   └── video2.mp4
│   ├── surg-002/
│   │   ├── video1.mp4
│   │   └── video2.mp4
│   └── ...
│── frames/
│── microclips/
└── labels.csv
```

In [ ]:
#| export
def get_frame_timestamps(video_path, interval):
    """
    Calculate the timestamps at which to extract frames based on the video duration and specified interval.
    
    Args:
        video_path (Path): Path to the input video file.
        interval (int): Interval in seconds to extract frames.
        
    Returns:
        List[float]: List of timestamps in seconds.
    """
    # Get video duration using ffprobe
    ffprobe_command = [
        'ffprobe',
        '-v', 'error',
        '-show_entries',
        'format=duration',
        '-of',
        'default=noprint_wrappers=1:nokey=1',
        str(video_path)
    ]
    result = subprocess.run(ffprobe_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    duration_str = result.stdout.strip()
    
    if not duration_str:
        print(f"Could not determine duration of video {video_path}")
        return []
    
    duration = float(duration_str)
    
    # Calculate timestamps
    timestamps = [round(i, 2) for i in np.arange(0, duration, interval)]
    return timestamps

In [ ]:
#| export
def split_video(video_path, output_dir, surgery_identifier, interval=1):
    """
    Extracts frames from a video at specified intervals using ffmpeg and saves them with timestamps in filenames.
    
    Args:
        video_path (Path): Path to the input video file.
        output_dir (Path): Directory to save the extracted frames.
        surgery_identifier (str): Identifier for the surgery (e.g., 'surg-001') to include in the frame filenames.
        interval (int): Interval in seconds to extract frames.
    
    Returns:
        None
    """
    video_name = video_path.stem
    output_dir.mkdir(parents=True, exist_ok=True)
    
    timestamps = get_frame_timestamps(video_path, interval)
    
    if not timestamps:
        return
    
    for timestamp in timestamps:
        output_filename = f"{surgery_identifier}_{video_name}_{timestamp:.2f}.jpg"
        output_file_path = output_dir / output_filename
        
        command = [
            'ffmpeg',
            '-ss', str(timestamp),
            '-i', str(video_path),
            '-frames:v', '1',
            '-q:v', '2',
            '-y',
            str(output_file_path)
        ]
        
        # Run the ffmpeg command
        subprocess.run(command, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

In [ ]:
#| export
def split_dataset(dataset_root_path='../temset', interval=1):
    """
    Process all videos in the dataset directory and extract frames to a frames folder using a single progress bar.
    
    Args:
        dataset_root_path (str): Path to the main dataset directory. Default is 'dataset'.
        interval (int): Interval in seconds to extract frames from videos.
    
        Returns:
            None
        """
    dataset_root_path = Path(dataset_root_path)
    videos_dir = dataset_root_path / 'videos'
    frames_dir = dataset_root_path / 'frames'
    
    # Get all video files in the dataset
    video_files = get_files(videos_dir, extensions='.mp4', recurse=True)
    
    with tqdm(total=len(video_files), desc="Extracting frames from all videos", unit='video') as pbar:
        for surgery_folder in videos_dir.ls():
            if surgery_folder.is_dir():
                print(f"Processing surgery: {surgery_folder.name}")
                video_files_in_folder = get_files(surgery_folder, extensions='.mp4')
    
                for video_path in video_files_in_folder:
                    split_video(video_path, frames_dir, surgery_folder.name, interval)
                    pbar.update(1)

In [ ]:
#| export
def main():
    """
    Main function to handle command-line arguments and execute the frame extraction process.
    """
    parser = argparse.ArgumentParser(description="Extract frames from videos in the dataset directory.")
    parser.add_argument('--dataset_root_path', type=str, default='../temset', help='Path to the dataset directory containing videos.')
    parser.add_argument('--interval', type=int, default=1, help='Interval in seconds for frame extraction.')
    
    args = parser.parse_args()
    process_dataset(args.dataset_root_path, args.interval)

if __name__ == "__main__":
    main()

In [ ]:
split_dataset()

In [ ]:
#| export
def main():
    """
    Main function to handle command-line arguments and execute the frame extraction process.
    """
    parser = argparse.ArgumentParser(description="Extract frames from videos in the dataset directory.")
    parser.add_argument('--dataset_root_path', type=str, default='../temset', help='Path to the dataset directory containing videos.')
    parser.add_argument('--interval', type=int, default=1, help='Interval in seconds for frame extraction. Use -1 to extract all frames.')

    args = parser.parse_args()
    split_dataset(args.dataset_root_path, args.interval)

if __name__ == "__main__":
    main()

In [8]:
#| hide
import nbdev; nbdev.nbdev_export()